<a href="https://colab.research.google.com/github/ououou92/ooo/blob/main/EX05_03_Reply_Multi_type_Messages.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!pip install -q flask pyngrok flask_ngrok line-bot-sdk

In [ ]:
# for colab
from google.colab import userdata
from pyngrok import ngrok
from flask_ngrok import run_with_ngrok
def ngrok_start():
    ngrok.set_auth_token(userdata.get('Authtoken'))
    ngrok.connect(5000)
    run_with_ngrok(app)


from flask import Flask, request, abort

from linebot.v3 import WebhookHandler
from linebot.v3.exceptions import InvalidSignatureError
from linebot.v3.webhooks import MessageEvent, TextMessageContent
from linebot.v3.messaging import (
    Configuration, ApiClient, MessagingApi,
    ReplyMessageRequest,
    TextMessage, StickerMessage, ImageMessage, VideoMessage, LocationMessage
)

app = Flask(__name__)

configuration = Configuration(access_token=userdata.get('CHANNEL_ACCESS_TOKEN'))
handler = WebhookHandler(userdata.get('Channel_secret'))

@app.route("/callback", methods=['POST'])
def callback():
    signature = request.headers['X-Line-Signature']
    body = request.get_data(as_text=True)
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        abort(400)
    return 'OK'

import google.generativeai as genai

genai.configure(api_key=userdata.get('GOOGLE_API_KEY'))
model = genai.GenerativeModel("gemini-2.0-flash")
def ask_gemini(question):
  response = model.generate_content(question)
  return response.text

@handler.add(MessageEvent, message=TextMessageContent)
def handle_message(event):
    with ApiClient(configuration) as api_client:
      line_bot_api = MessagingApi(api_client)

      action = event.message.text
      if action == 'sticker':
            reply = StickerMessage(
                package_id="446",
                sticker_id="1989"
                )
      elif action == 'image':
            reply = ImageMessage(
                original_content_url='https://i.pinimg.com/736x/88/f5/f2/88f5f220d159a5d306314c4223295f6b.jpg',
                preview_image_url='https://www.pinterest.com/search/pins/?q=Yuqi&rs=rs&source_id=rs_5Cfs15cs&eq=&etslf=1895'
                )
      elif action == 'video':
            reply = VideoMessage(
                original_content_url='https://cdn.pixabay.com/video/2023/03/08/153821-806526710_tiny.mp4',
                preview_image_url='https://cdn.pixabay.com/photo/2024/05/23/07/17/woman-8782450_1280.jpg'
            )
      elif action == 'location':
            reply = LocationMessage(
              title = 'Taipei 101',
              address = '110台北市信義區信義路5段7號',
              latitude = 25.047924,
              longitude = 121.517081
            )
      else:
        response = ask_gemini(event.message.text)
        reply = TextMessage(text=response)

      line_bot_api.reply_message(
          ReplyMessageRequest(
            reply_token=event.reply_token,
            messages=[
                reply
                ]
            )
        )



ngrok_start() # for colab
if __name__ == "__main__":
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://1299-34-81-237-127.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [06/May/2025 10:14:28] "GET / HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [06/May/2025 10:14:30] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [06/May/2025 10:14:42] "POST /callback HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [06/May/2025 10:14:52] "POST /callback HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [06/May/2025 10:15:04] "POST /callback HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [06/May/2025 10:15:11] "POST /callback HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [06/May/2025 10:15:19] "POST /callback HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [06/May/2025 10:15:28] "POST /callback HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [06/May/2025 10:15:39] "POST /callback HTTP/1.1" 200 -
